# Write one zarr, then append to it.
Doesnt work.


In [4]:
import os
import gcsfs
import fsspec
import json
import xarray as xr
import sys
sys.path.append("/home/jovyan/XApRES/src/XApRES/")
import ApRESDefs
import zarr
from tqdm.notebook import trange, tqdm

Get a mapper to a new location in the bucket to write the zarr store

In [5]:
with open('../../secrets/ldeo-glaciology-bc97b12df06b.json') as token_file:
    token = json.load(token_file)

filename = 'gs://ldeo-glaciology/GL_apres_2022_zarr_test_small_5' 
mapper = fsspec.get_mapper(filename, mode='w', token=token)


Produce and write the first ApRES xarray, just reading 2 bursts from each file (there are actually 96 in each file).

In [6]:
xa = ApRESDefs.xapres(max_range=1400)
xa.load_all(directory='gs://ldeo-glaciology/GL_apres_2022/A101', 
            remote_load = True,
            file_numbers_to_process=[3], bursts_to_process=[0,1])

ds = xa.data
ds.attrs = []
ds.to_zarr(mapper, mode='w', consolidated=False);

100%|██████████| 2/2 [00:02<00:00,  1.48s/it]


Loop through the dat files up for file 31 and append to the zarr. 



In [11]:
xa = ApRESDefs.xapres(max_range=1400)
xa.list_files(directory='gs://ldeo-glaciology/GL_apres_2022/A101', 
            remote_load = True)

for file in trange(4,10): # len(xa.dat_filenames)
    xa = ApRESDefs.xapres(max_range=1400)
    xa.load_all(directory='gs://ldeo-glaciology/GL_apres_2022/A101', 
            remote_load = True,
            file_numbers_to_process=[file],  bursts_to_process=[0,1])
    
    ds = xa.data
    ds.attrs = []
    ds.to_zarr(mapper, mode='a', append_dim="time", consolidated=False)
    
    #zarr.consolidate_metadata(mapper) 

    
ds_reloaded = xr.open_dataset(filename, engine='zarr', consolidated=False)
ds_reloaded

  0%|          | 0/6 [00:00<?, ?it/s]


100%|██████████| 2/2 [00:02<00:00,  1.34s/it]

100%|██████████| 2/2 [00:02<00:00,  1.40s/it]

100%|██████████| 2/2 [00:02<00:00,  1.32s/it]

100%|██████████| 2/2 [00:02<00:00,  1.41s/it]

100%|██████████| 2/2 [00:02<00:00,  1.35s/it]

100%|██████████| 2/2 [00:02<00:00,  1.41s/it]


<xarray.Dataset>
Dimensions:          (attenuator_setting_pair: 2, time: 2, chirp_time: 40001,
                      chirp_num: 20, profile_range: 6658)
Coordinates:
    AFGain           (attenuator_setting_pair) int64 ...
    attenuator       (attenuator_setting_pair) float64 ...
    burst_number     (time) int64 ...
  * chirp_num        (chirp_num) int64 0 1 2 3 4 5 6 7 ... 13 14 15 16 17 18 19
  * chirp_time       (chirp_time) timedelta64[ns] 00:00:00 ... 00:00:01
    filename         (time) <U83 ...
  * profile_range    (profile_range) float64 0.0 0.2103 ... 1.4e+03 1.4e+03
  * time             (time) datetime64[ns] 2022-05-27T15:06:30 2022-05-27T15:...
Dimensions without coordinates: attenuator_setting_pair
Data variables:
    battery_voltage  (time) float64 ...
    chirp            (time, chirp_time, chirp_num, attenuator_setting_pair) float64 ...
    latitude         (time) float64 ...
    longitude        (time) float64 ...
    profile          (time, profile_range, chirp_num, attenuator_setting_pair) complex128 ...
    temperature_1    (time) float64 ...
    temperature_2    (time) float64 ...

We should see a zarr store with a time dimension length of around 2*(1+30-4) = 54, but it is only 2.

In [ ]:
zarr.consolidate_metadata(mapper) 
ds_reloaded = xr.open_dataset(filename, engine='zarr', consolidated=False)
ds_reloaded

<xarray.Dataset>
Dimensions:          (attenuator_setting_pair: 2, time: 2, chirp_time: 40001,
                      chirp_num: 20, profile_range: 6658)
Coordinates:
    AFGain           (attenuator_setting_pair) int64 ...
    attenuator       (attenuator_setting_pair) float64 ...
    burst_number     (time) int64 ...
  * chirp_num        (chirp_num) int64 0 1 2 3 4 5 6 7 ... 13 14 15 16 17 18 19
  * chirp_time       (chirp_time) timedelta64[ns] 00:00:00 ... 00:00:01
    filename         (time) <U83 ...
  * profile_range    (profile_range) float64 0.0 0.2103 ... 1.4e+03 1.4e+03
  * time             (time) datetime64[ns] 2022-05-27T15:06:30 2022-05-27T15:...
Dimensions without coordinates: attenuator_setting_pair
Data variables:
    battery_voltage  (time) float64 ...
    chirp            (time, chirp_time, chirp_num, attenuator_setting_pair) float64 ...
    latitude         (time) float64 ...
    longitude        (time) float64 ...
    profile          (time, profile_range, chirp_num, attenuator_setting_pair) complex128 ...
    temperature_1    (time) float64 ...
    temperature_2    (time) float64 ...

using curl (as advides in the [issue](https://github.com/pydata/xarray/issues/5878) shows a length of 4.

In [16]:
!curl https://storage.googleapis.com/ldeo-glaciology/GL_apres_2022_zarr_test_small_1/time/.zarray

{
    "chunks": [
        2
    ],
    "compressor": {
        "blocksize": 0,
        "clevel": 5,
        "cname": "lz4",
        "id": "blosc",
        "shuffle": 1
    },
    "dtype": "<i8",
    "fill_value": null,
    "filters": null,
    "order": "C",
    "shape": [
        4
    ],
    "zarr_format": 2
}

# Notes on a longer loop that failed after a while with an error. 
When I tried to run the full loop (all bursts and all dat files) it failed after several dat files have been loaded: 
```
File /srv/conda/envs/notebook/lib/python3.9/site-packages/xarray/backends/zarr.py:482, in ZarrStore.get_dimensions(self)
    480     for d, s in zip(dim_names, v.shape):
    481         if d in dimensions and dimensions[d] != s:
--> 482             raise ValueError(
    483                 f"found conflicting lengths for dimension {d} "
    484                 f"({s} != {dimensions[d]})"
    485             )
    486         dimensions[d] = s
    487 return dimensions

ValueError: found conflicting lengths for dimension time (564 != 658)
xa.dat_filenames[len(xa.dat_filenames)-1]
```
This, i think, is down to caching, because there is a difference between the length of the array you get when you use
`ds_reloaded = xr.open_dataset('gs://ldeo-glaciology/GL_apres_2022_zarr_test_large_3', engine='zarr', consolidated=False, chunks='auto')
ds_reloaded`
658

and

` ds_reloaded = xr.open_dataset('gs://ldeo-glaciology/GL_apres_2022_zarr_test_large_3', engine='zarr', consolidated=True, chunks='auto')
ds_reloaded`
564